In [ ]:
from mvodolagin_personal_imports import *

load_dotenv()

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from urllib.parse import urljoin
import scrapy


class MySpider(scrapy.Spider):
    name = "carolhouse_spider"
    start_urls = ["http://www.carolhouse.com"]

    def parse(self, response, **kwargs):
        # Look for 'Contact Us' link and yield a request with a callback
        # Note: You might need to adjust the selector based on the actual site structure
        contact_us_link = response.xpath('//a[contains(text(), "Contact Us")]/@href').get()
        if contact_us_link:
            yield response.follow(contact_us_link, self.parse_contact, meta={"is_contact_page": True})

        # Find item links - adjust the selector to match your target site's structure
        item_links = response.xpath('//a[contains(@href, "item")]/@href').getall()
        for link in item_links:
            yield response.follow(link, self.parse_item)

        # Follow other potential links to pages like 'Contact Us' or item pages.
        # This is a simplistic approach; you may need to refine how you select links to follow.
        for href in response.xpath("//a/@href").getall():
            yield response.follow(href, self.parse)

    def parse_contact(self, response):
        # Process the 'Contact Us' page here
        # For example, extract the email address or form action
        # Remember to check response.meta['is_contact_page'] if needed
        yield {
            "url": response.url,
            "is_contact_page": response.meta.get("is_contact_page", False),
            # Add any other item fields you want to extract from the contact page
        }

    def parse_item(self, response):
        # Extract item details such as description and images
        # Adjust selectors to match the structure of the item pages you're targeting
        yield {
            "url": response.url,
            "description": response.xpath('//div[contains(@class, "description")]/text()').get(),
            "images": response.xpath("//img/@src").getall(),
            # You might need to join image URLs with the base URL
            "images": [urljoin(response.url, img) for img in response.xpath("//img/@src").getall()],
            # Add any other fields you want to extract
        }

In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

# Initialize and run the spider
# Note: You might need to adjust get_project_settings() based on your Scrapy project setup or provide custom settings
process = CrawlerProcess(get_project_settings())

process.crawl(MySpider)
process.start()

In [ ]:
# from twisted.internet import asyncioreactor
# asyncioreactor.install()

from scrapy import Selector
from scrapy.http import Request
from twisted.internet import defer, reactor
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerRunner

# Initialize the crawler runner with your project settings
runner = CrawlerRunner(get_project_settings())


In [ ]:
@defer.inlineCallbacks
def fetch_page(url):
    # The callback function to process the downloaded response
    def parse(response):
        return response
    
    # Schedule a request and return the response when it's done
    yield runner.crawl(MySpider, start_urls=[url], parse=parse)

# Example usage to fetch and store the response for inspection
response = await fetch_page('https://www.carolhouse.com')


In [ ]:
fetch_page('https://www.carolhouse.com')

In [ ]:
spider = MySpider()

In [ ]:
qq = spider.parse('https://www.carolhouse.com')

In [ ]:
qq.__next__()

In [ ]:
import requests

In [ ]:
url = "https://www.carolhouse.com/living-room/cabinets/room-type.aspx"

response = requests.get(url)

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(response.text)